In [26]:
import mysql.connector
import cv2
import numpy as np
import face_recognition
from datetime import datetime, timedelta, time

# Dictionary to keep track of the last attendance time for each student
last_attendance_time = {}

# Create a global database connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="attendancedb"
)

def get_current_course():
    cursor = None
    try:
        cursor = db.cursor()
        now = datetime.now()
        current_time = now.time()
        day = now.weekday()  # Monday is 0 and Sunday is 6

        sql = """
        SELECT course_code, start_time, end_time, venue FROM timetable 
        WHERE day = %s AND start_time <= %s AND end_time >= %s
        """
        cursor.execute(sql, (day, current_time, current_time))
        result = cursor.fetchone()

        if result:
            print(f"Current course: {result[0]}, start time: {result[1]}, end time: {result[2]}, venue: {result[3]}")
            # Ensure start_time and end_time are datetime.time objects
            start_time = result[1] if isinstance(result[1], time) else (datetime.min + result[1]).time()
            end_time = result[2] if isinstance(result[2], time) else (datetime.min + result[2]).time()
            return result[0], start_time, end_time, result[3]  # Return course_code, start_time, end_time, and venue
        else:
            print("No current course found")
            return None, None, None, None

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None, None, None, None

    finally:
        if cursor is not None:
            cursor.fetchall()  # Ensure all results are fetched before closing the cursor
            cursor.close()

def is_student_enrolled(student_no, course_code):
    cursor = None
    try:
        cursor = db.cursor()
        sql = "SELECT 1 FROM course_reg WHERE student_no = %s AND course_code = %s"
        cursor.execute(sql, (student_no, course_code))
        result = cursor.fetchone()
        return result is not None

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return False

    finally:
        if cursor is not None:
            cursor.fetchall()  # Ensure all results are fetched before closing the cursor
            cursor.close()

def markAttendance(student_no, record_type, course_code, status):
    cursor = None
    try:
        cursor = db.cursor()

        now = datetime.now()
        dtString = now.strftime('%Y-%m-%d %H:%M:%S')

        # Insert into the MySQL database
        sql = "INSERT INTO attendance (date, time, type, student_no, course_code, status) VALUES (%s, %s, %s, %s, %s, %s)"
        val = (now.date(), dtString, record_type, student_no, course_code, status)
        cursor.execute(sql, val)
        db.commit()

        print(f"Attendance {record_type} marked for {student_no} at {dtString} for course_code {course_code} with status {status}")

        # Update the last attendance time
        last_attendance_time[student_no] = (now, record_type)

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor is not None:
            cursor.fetchall()  # Ensure all results are fetched before closing the cursor
            cursor.close()

def markAbsent(student_no, course_code):
    cursor = None
    try:
        cursor = db.cursor()

        now = datetime.now()
        dtString = now.strftime('%Y-%m-%d %H:%M:%S')

        # Insert into the MySQL database
        sql = "INSERT INTO attendance (date, student_no, course_code) VALUES (%s, %s, %s)"
        val = (now.date(), student_no, course_code)
        cursor.execute(sql, val)
        db.commit()

        print(f"Absence recorded for {student_no} at {now.date()} for course_code {course_code}")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor is not None:
            cursor.close()

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def load_images_from_mysql():
    images = []
    student_numbers = []
    cursor = None
    try:
        cursor = db.cursor()

        # Query to select images from MySQL table
        sql = "SELECT student_no, image FROM images"
        cursor.execute(sql)

        for (student_no, image_blob) in cursor.fetchall():
            # Convert BLOB data to image array usable by OpenCV
            image_array = np.frombuffer(image_blob, dtype=np.uint8)
            img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            if img is not None:
                images.append(img)
                student_numbers.append(student_no)

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if cursor is not None:
            cursor.fetchall()  # Ensure all results are fetched before closing the cursor
            cursor.close()

    return images, student_numbers

def get_expected_venue(course_code):
    cursor = None
    try:
        cursor = db.cursor()

        sql = "SELECT venue FROM timetable WHERE course_code = %s"
        cursor.execute(sql, (course_code,))
        result = cursor.fetchone()

        if result:
            return result[0]  # Return the venue

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

    finally:
        if cursor is not None:
            cursor.fetchall()  # Ensure all results are fetched before closing the cursor
            cursor.close()

    return None

# Load images and student numbers from MySQL
images, student_numbers = load_images_from_mysql()

if not images:
    print("No images found in the database.")
else:
    encodeListKnown = findEncodings(images)
    print('Encoding Complete')

    # Fetch the current course once before starting the loop
    course_code, start_time, end_time, venue = get_current_course()
    expected_venue = get_expected_venue(course_code) if course_code else None

    cap = cv2.VideoCapture(0)

    while True:
        success, img = cap.read()
        if not success:
            print("Failed to capture image")
            break

        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

        for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            matchIndex = np.argmin(faceDis)

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            if matches[matchIndex]:
                student_no = student_numbers[matchIndex]
                now = datetime.now()

                # Determine if clocking in or clocking out
                if student_no in last_attendance_time:
                    last_time, last_type = last_attendance_time[student_no]
                    if last_type == "ClockIn":
                        record_type = "ClockOut"
                    else:
                        record_type = "ClockIn"
                else:
                    record_type = "ClockIn"

                # Check if the student's last attendance was recorded more than 5 minutes ago
                if student_no not in last_attendance_time or now - last_attendance_time[student_no][0] > timedelta(minutes=5):
                    if course_code and expected_venue:
                        # Verify the venue
                        if venue == expected_venue:
                            # Convert start_time to a datetime object for comparison
                            class_start_time = datetime.combine(datetime.today(), start_time)

                            # Allow scanning to start up to 10 minutes before class starts
                            if (now - class_start_time).total_seconds() >= -600:  # 10 minutes in seconds
                                if now.time() <= class_start_time.time():
                                    status = "On time"
                                elif now.time() <= (class_start_time + timedelta(minutes=10)).time():
                                    status = "Late"
                                else:
                                    status = "Too Late"

                                if is_student_enrolled(student_no, course_code):
                                    markAttendance(student_no, record_type, course_code, status)
                                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle for known faces
                                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                                    cv2.putText(img, f"{student_no} ({status})", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                                else:
                                    print(f"Student {student_no} is not enrolled in course {course_code}")
                                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle for known faces
                                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                                    cv2.putText(img, f"{student_no}", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                            else:
                                print(f"Incorrect venue. Expected: {expected_venue}, Scanned: {venue}")
                        else:
                            print("Venue mismatch or course not found.")
                    else:
                        print("No current course or venue data available.")
                else:
                    print(f"Student {student_no} is taking short break")
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle for known faces
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, f"{student_no}", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            else:
                # Draw rectangle around unknown faces
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red rectangle for unknown faces
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 0, 255), cv2.FILLED)
                cv2.putText(img, "Unknown", (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow('Webcam', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Mark absent students after the class ends
for student_no in student_numbers:
    if student_no not in last_attendance_time and is_student_enrolled(student_no, course_code):
        markAbsent(student_no, course_code)

# Close the database connection when done
db.close()


Encoding Complete
Current course: mcs306, start time: 0:00:00, end time: 10:30:00, venue: Lab1
Venue mismatch or course not found.
Venue mismatch or course not found.
Absence recorded for 202080162 at 2024-08-30 for course_code mcs306
Absence recorded for 202280123 at 2024-08-30 for course_code mcs306
Absence recorded for 202080165 at 2024-08-30 for course_code mcs306
Absence recorded for 202080175 at 2024-08-30 for course_code mcs306
Absence recorded for 202180167 at 2024-08-30 for course_code mcs306
Absence recorded for 202280121 at 2024-08-30 for course_code mcs306
Absence recorded for 202180167 at 2024-08-30 for course_code mcs306
Absence recorded for 202080175 at 2024-08-30 for course_code mcs306
Absence recorded for 202080167 at 2024-08-30 for course_code mcs306
Absence recorded for 202080165 at 2024-08-30 for course_code mcs306
